In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 1.15 s


In [3]:
final_pt_df2['final_bin'].value_counts()

C_neg/A_partial    7867
C_neg/A_full       7401
C_pos/A_full       2438
C_pos/A_partial    1927
Name: final_bin, dtype: int64

time: 6.98 ms


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

#NOTE ON MY DF NAMING CONVENTION:
origionally when I coded this workbook, it was for 72 hour timewindows, so every dataframe had _72 at the end. this was changed on 6/5/19 and was made more generalizable by finding name of each corresponding df in the df list and using this variable.

In [4]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df, save_boolean


time: 741 µs


### begin pipeline:

# changing my code structure to be a dictionary of dataframes

In [5]:
#folder to save files to:
save_path= str(repository_path)+'/data/cleaned/'
#folder=None


def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_prepped.csv' %(date, df_name)))


time: 6.84 ms


In [6]:
save_path+'%s/'%(folder)

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/72_hr_window/'

time: 1.64 ms


In [7]:
##folder with all clinical variable csv's
allFiles = glob.glob(str(repository_path)+ '/data/raw/%s/'%(folder) + "{}_*.csv".format(date))

time: 2.42 ms


In [8]:
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_rrt_merged.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_labs_all_nosummary.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_cancer_elix.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_bg_all_nosummary.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_pt_info.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_vitals_all_nosummary.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_gcs.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/72_hr_window/10102019_sum_elix.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-op

time: 2.42 ms


In [9]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 20.9 s


In [10]:
df_list

['rrt_merged',
 'labs_all_nosummary',
 'cancer_elix',
 'bg_all_nosummary',
 'pt_info',
 'vitals_all_nosummary',
 'gcs',
 'sum_elix',
 'sofa',
 'ventcategory',
 'uti_all',
 'vaso_dose',
 'heightfirstday',
 'weightfirstday']

time: 2.27 ms


In [11]:
#assigning the appropriate name to each df in a flexible way

indices = [i for i, s in enumerate(df_list) if 'bg' in s]
bg_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'cancer' in s]
cancer_elix_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'uti' in s]
uti_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vent' in s]
vent_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vitals' in s]
vitals_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vaso' in s]
vaso_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'pt_info' in s]
pt_info_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'gcs' in s]
gcs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'sum_elix' in s]
sum_elix_df = df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'sofa' in s]
sofa_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'weight' in s]
weight_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'labs' in s]
labs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'height' in s]
height_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'rrt' in s]
rrt_df= df_list[indices[0]]

time: 109 ms


In [12]:
#adding a t_0 to each df that doesn't currently have it

for element in df_list:
    #print(element,':',list(dfs[element]))
    if ('t_0' in list(dfs[element]))==False and 'icustay_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','t_0']], how='left')
    elif ('t_0' in list(dfs[element]))==False and 'hadm_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['hadm_id','t_0']], how='left')
    else:
        print("false")


false
false
false
false
false
false
false
false
false
false
time: 112 ms


In [13]:
for element in df_list:
    print(element,':',list(dfs[element]))

rrt_merged : ['icustay_id', 't_0', 'first_charttime', 'rrt', 'uom']
labs_all_nosummary : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'label', 'valuenum', 'uom', 't_0', 'delta']
cancer_elix : ['subject_id', 'hadm_id', 'icustay_id', 'value', 'label', 'delta', 'uom', 't_0']
bg_all_nosummary : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'label', 'valuenum', 'value', 'valueuom', 'unique_var', 't_0', 'delta']
pt_info : ['icustay_id', 'subject_id', 't_0', 'label', 'value', 'delta', 'uom']
vitals_all_nosummary : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'valueuom', 'vitalid', 'valuenum', 't_0', 'delta']
gcs : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'mingcs', 'gcsmotor', 'gcsverbal', 'gcseyes', 'endotrachflag', 'icu_admit', 't_0', 'approx_charttime', 'admit_plus_day', 'delta', 'uom']
sum_elix : ['subject_id', 'hadm_id', 'icustay_id', 'value', 'label', 'delta', 'uom', 't_0']
sofa : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'sofa', 'respiration', 'pao2fio2

In [14]:
from parameters import time_var, value_fill, delta_fill, uom_fill
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var=time_var,#'t_0', #
               value_fill=value_fill,#0,
               delta_fill=delta_fill,# pd.to_timedelta('0 days'),
               uom_fill=uom_fill):#'y/n'):

    """
    description: collapses (binarizes) a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have a non NA value in the clinical time window y/n? 
    
    label_fill: the variable name in the label column of the specified dataframe that will considered for y/n value within timewindow. if any non NA value is present
        it will be considered positive.
    pt: the by patient spreadsheet be be used to supply patient information.
    time_var: the variable used to create the time window of interest.
    value_fill: the variable value that missing values will be filled if the value is not present (default =0) in the origional dataset
    delta_fill: the time delta value that will be filled in if a patient doesn't have any instances of the label_fill.  
    uom_fill: fills in the unit of measurement to this for missing values.
    
    returns a flat 1 row per icustay_id of 1 or 0 if any value was present for the patient.
    """
    
    yn_df = pd.merge(pt[['icustay_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)


time: 31.2 ms


## vaso dose

In [17]:
# #renaming starttime to charttime and dropping endtime

dfs[vaso_df]= dfs[vaso_df].rename(
    columns={'starttime':'charttime','label':'vaso_type'})

time: 24.4 ms


In [18]:
len(dfs[vaso_df])

59474

time: 1.8 ms


In [19]:
dfs[vaso_df].head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
8,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
22,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000
23,200952.0,2139-09-24 21:54:00,2139-09-25 13:17:00,0.201337,7.325205,mg,mcg/kg/min,norepinephrine,2139-09-23,1 days 21:54:00.000000000
34,200952.0,2139-09-23 15:06:00,2139-09-24 00:51:00,2.509001,43.660095,mg,mcg/kg/min,phenylephrine,2139-09-23,0 days 15:06:00.000000000


time: 14.1 ms


In [20]:
#removing units/hour because that is a different use of vasopressin
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'units/hour',:]
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'Uhr',:]

time: 17.2 ms


In [21]:
len(dfs[vaso_df])

59474

time: 2.89 ms


In [22]:
dfs[vaso_df]['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin', 'units/min'], dtype=object)

time: 6.33 ms


#### removing outliers/extreme values

In [23]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:
def vaso_outlier_removal(df):
    test_group=(dfs[vaso_df][['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
         .groupby(['vaso_type','rate_uom'])
             )

    dfs[vaso_df]['std']=test_group.transform(lambda x : x.std())
    dfs[vaso_df]['mean']=test_group.transform(lambda x : x.mean())

    normal_high_value= pd.DataFrame({
        'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
        'high_value': [40, 20, 0.5, 1, 0.1, 2] #highest values one might expect to see in a clinic, ie above this is likely erroneous
    }) #found from literature, see notes

    dfs[vaso_df] = pd.merge(dfs[vaso_df], normal_high_value, left_on='vaso_type', right_on='vaso_type')

    vaso_dose_72_rmout =(dfs[vaso_df][
        ~((dfs[vaso_df]['vaso_rate'] > dfs[vaso_df]['high_value']) & ((dfs[vaso_df]['vaso_rate']-dfs[vaso_df]['mean'])>= (3*dfs[vaso_df]['std'])))
        ])
    #ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean
    return(vaso_dose_72_rmout)

time: 23.2 ms


In [24]:
dfs[vaso_df]= vaso_outlier_removal(dfs[vaso_df])
len(dfs[vaso_df]) #52976 ->49340

59135

time: 182 ms


In [25]:
dfs[vaso_df]['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 5.62 ms


In [26]:
#standardizing names, dropping unneeded columns for analysis
dfs[vaso_df]= dfs[vaso_df].drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
dfs[vaso_df]= dfs[vaso_df].rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})
dfs[vaso_df]['label'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 64.8 ms


In [27]:
dfs[vaso_df].head()

,icustay_id,charttime,endtime,value,uom,label,t_0,delta
0,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
1,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
2,200977.0,2166-12-02 16:59:00,2166-12-03 05:50:00,0.030015,mcg/kg/min,epinephrine,2166-12-02,0 days 16:59:00.000000000
3,201005.0,2169-10-25 17:25:00,2169-10-25 18:00:00,0.020000,mcgkgmin,epinephrine,2169-10-25,0 days 17:25:00.000000000
4,201005.0,2169-10-25 18:30:00,2169-10-25 19:00:00,0.020000,mcgkgmin,epinephrine,2169-10-25,0 days 18:30:00.000000000


time: 11.4 ms


In [28]:
epinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='epinephrine']
norepinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='norepinephrine']
phenylephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='phenylephrine']
vasopressin_df= dfs[vaso_df][dfs[vaso_df]['label']=='vasopressin']
dopamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dopamine']
dobutamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dobutamine']


time: 39.6 ms


In [29]:
#(could be transfered to a different spdsheet for collapsing values)
#y/n convert, seperating out vaso_dose into 6 constitutient dataframes, and for each am collapsing values down to does pt have in time window y/n? 

epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)

time: 985 ms


In [30]:
save_df(epinephrine_df, df_name='epinephrine')

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/
time: 409 ms


In [31]:
# pd.DataFrame(epinephrine_df).to_csv(Path(
#     save_path+ '/%s_epinephrine_prepped.csv' %(date)))

save_df(epinephrine_df, df_name='epinephrine')
del(epinephrine_df)

save_df(norepinephrine_df, df_name='norepinephrine')
del(norepinephrine_df)

save_df(phenylephrine_df, df_name='phenylephrine')
del(phenylephrine_df)

save_df(vasopressin_df, df_name='vasopressin')
del(vasopressin_df)

save_df(dopamine_df, df_name='dopamine')
del(dopamine_df)

save_df(dobutamine_df, df_name='dobutamine')
del(dobutamine_df)

del(dfs[vaso_df])


time: 4.04 s


## ELIX

In [32]:
#convert cancer elix to y/n:
dfs[cancer_elix_df]= yn_convert(dfs[cancer_elix_df], label_fill=0, pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
save_df(dfs[cancer_elix_df], df_name='cancer_elix')
del(dfs[cancer_elix_df])

save_df(dfs[sum_elix_df], 'sum_elix')
del(dfs[sum_elix_df])

time: 775 ms


## vitals -

In [15]:
dfs[vitals_df].head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum,t_0,delta
0,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0,2149-11-10,1 days 00:00:00.000000000
1,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0,2149-11-10,1 days 19:00:00.000000000
2,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0,2149-11-10,1 days 19:45:00.000000000
3,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0,2149-11-10,1 days 20:00:00.000000000
14,9,150750,220597,2149-11-12 14:00:00,mmHg,MeanArtPress,89.0,2149-11-10,2 days 14:00:00.000000000


time: 22.4 ms


In [15]:
dfs[vitals_df] = dfs[vitals_df].rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['label'].notnull(),:]#.count() #removing null values

dfs[vitals_df].loc[dfs[vitals_df].loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='RespRate') & 
    (dfs[vitals_df]['uom']=='bpm'),'uom']='insp/min'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC') & 
    (dfs[vitals_df]['uom']=='?C'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='Deg. F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='?F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF'),'label']='temperature'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC'),'label']='temperature'

time: 21.4 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

### most likely erroneous value removal

In [16]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'icustay_id']==228393.0) &
                         (dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99999), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99998), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='DiasBP') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='HeartRate') &
                         (dfs[vitals_df].loc[:,'value'].between(1,29)), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='RespRate') &
                         (dfs[vitals_df].loc[:,'value']<4), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='SysBP') &
                         (dfs[vitals_df].loc[:,'value']<40), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='TempC') &
                         (dfs[vitals_df].loc[:,'value']<28), 'value'])=np.nan

time: 3.39 s


In [17]:
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['value'].notnull(),:]#.count() 


time: 5.95 s


In [18]:
### saving spo2 for later use in bloodgas 11/25/19
spo2=dfs[vitals_df][dfs[vitals_df]['label']=='SpO2'].copy()

time: 870 ms


In [20]:
save_df(dfs[vitals_df], 'vitals')
del(dfs[vitals_df])


time: 78.3 ms


# labs -

In [15]:
dfs[labs_df]= dfs[labs_df].rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 174 ms


In [16]:
dfs[labs_df].groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL        7257
ANION GAP    mEq/L      33825
BANDS        %           2860
BICARBONATE  mEq/L      34652
BILIRUBIN    mg/dL      11243
BUN          mg/dL      34984
CHLORIDE     mEq/L      40705
CREATININE   mg/dL      35129
GLUCOSE      mg/dL      50811
HEMATOCRIT   %          49845
HEMOGLOBIN   g/dL       40784
LACTATE      mmol/L     28282
PLATELET     K/uL       33054
POTASSIUM    mEq/L      53467
PT           SECONDS    25239
PTT          sec        26141
SODIUM       mEq/L      43014
WBC          K/uL       31582
Name: uom, dtype: int64

time: 176 ms


### most likely erroneous value removal

In [17]:
#summary value removal- was explored and coded adhoc, difficult to automate
(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==261887) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==3.4),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==236290) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==11.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==292769) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==28.1),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==298457) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.8),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==234174) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.7),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==42.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==22.8),'value'])=np.nan

time: 287 ms


In [18]:
dfs[labs_df]= dfs[labs_df].loc[dfs[labs_df].loc[:,'value'].notnull(),:] #removing null values


time: 49.3 ms


In [19]:
dfs[labs_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,value,uom,t_0,delta
1,4,185777,294638,2191-03-16 05:42:00,ANION GAP,15.0,mEq/L,2191-03-16,0 days 05:42:00.000000000
9,4,185777,294638,2191-03-16 05:42:00,BICARBONATE,21.0,mEq/L,2191-03-16,0 days 05:42:00.000000000
17,4,185777,294638,2191-03-16 05:42:00,BILIRUBIN,1.9,mg/dL,2191-03-16,0 days 05:42:00.000000000
19,4,185777,294638,2191-03-16 05:42:00,CHLORIDE,108.0,mEq/L,2191-03-16,0 days 05:42:00.000000000
27,4,185777,294638,2191-03-16 05:42:00,CREATININE,0.5,mg/dL,2191-03-16,0 days 05:42:00.000000000


time: 15.3 ms


In [20]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
dfs[labs_df]= dfs[labs_df].loc[~dfs[labs_df].loc[:,'label'].isin(unwanted_values),:]

time: 44.5 ms


# factorizing bands start 
* converting bands into a categorical variable since it is very sparse

In [44]:

#df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_band(df,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    """
    binarizing bands value since it's extremely sparse. Wrote a new function instead of adjusting yn_convert a while back. on my todo to remove this and add functionality to regular yn_conmert function. 
    """
    
    yn_df = pd.merge(pt[['icustay_id','hadm_id','subject_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 33.3 ms


In [45]:
band_df=dfs[labs_df][dfs[labs_df]['label']=='BANDS']
max_bands=band_df.loc[band_df.groupby('icustay_id', as_index=False)['value'].idxmax(),:]
del(band_df)

time: 595 ms


In [46]:
band_cat=yn_convert_band(df=max_bands,           
                    label_fill="BANDS",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 115 ms


In [47]:
#drop bands from lab_df
dfs[labs_df]=dfs[labs_df].drop(dfs[labs_df][dfs[labs_df]['label']=='BANDS'].index)

##dropping charttime, may be problematic later.
dfs[labs_df]=dfs[labs_df].drop('charttime', axis=1)

#appending categorical bands 
dfs[labs_df]=dfs[labs_df].append(band_cat)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


time: 601 ms


In [48]:
dfs[labs_df].loc[dfs[labs_df]['label']=='BANDS','value'].value_counts()

absent    16546
<10        1982
>10        1105
Name: value, dtype: int64

time: 53.2 ms


In [49]:
# pd.DataFrame(dfs[labs_df]).to_csv(Path(
#     save_path+'/%s_labs_prepped.csv' %(date)))

save_df(dfs[labs_df], 'labs')
del(dfs[labs_df])

time: 5.98 s


## vent category -

In [19]:
dfs[vent_df]['icustay_id'].nunique()
#13978 patients with someform of vent data. 

19471

time: 17.1 ms


In [20]:
dfs[vent_df]['uom']='mech/O2/none category'
dfs[vent_df].head()

,icustay_id,day,value,uom,t_0
0,200001.0,0,Oxygen,mech/O2/none category,2181-11-26
1,200001.0,1,Oxygen,mech/O2/none category,2181-11-26
2,200001.0,2,Oxygen,mech/O2/none category,2181-11-26
3,200003.0,0,Mech,mech/O2/none category,2199-08-04
4,200003.0,1,Mech,mech/O2/none category,2199-08-04


time: 15.4 ms


In [21]:
dfs[vent_df]=dfs[vent_df].rename(index=str, columns={'day':'delta'})
dfs[vent_df]['label']='vent_recieved'
dfs[vent_df]['delta']=pd.to_timedelta(dfs[vent_df]['delta'], unit='d')
#dfs[vent_df]= dfs[vent_df].drop(columns=['day'], axis=1) #removing day column

time: 51.7 ms


In [22]:
dfs[vent_df].head()

,icustay_id,delta,value,uom,t_0,label
0,200001.0,0 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
1,200001.0,1 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
2,200001.0,2 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
3,200003.0,0 days,Mech,mech/O2/none category,2199-08-04,vent_recieved
4,200003.0,1 days,Mech,mech/O2/none category,2199-08-04,vent_recieved


time: 11.7 ms


In [23]:
#collapsing into 1 column for N days

time: 420 µs


In [24]:
def vent_day_collapser(x):
    """
    collapsing the ventilation days into a single value. Mech> Oxygen > None. 
    """
    
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
    return(x)

time: 4.79 ms


In [25]:
#collapsing all days into the worst day. 
ventcategory_1day_df= dfs[vent_df].copy()
ventcategory_1day_df['value']=ventcategory_1day_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_df= ventcategory_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
ventcategory_1day_df= ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort, fail safe mesure
ventcategory_1day_df.head()

,icustay_id,delta,value,uom,t_0,label
0,200001.0,0 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
3,200003.0,0 days,Mech,mech/O2/none category,2199-08-04,vent_recieved
6,200012.0,0 days,Oxygen,mech/O2/none category,2153-12-23,vent_recieved
7,200014.0,0 days,Mech,mech/O2/none category,2105-02-17,vent_recieved
9,200019.0,0 days,Mech,mech/O2/none category,2178-07-09,vent_recieved


time: 29.6 s


In [26]:
ventcategory_1day_df['value'].value_counts()

Mech      10156
Oxygen     8369
None        946
Name: value, dtype: int64

time: 5.88 ms


In [25]:
save_df(ventcategory_1day_df, 'ventcategory')
del(ventcategory_1day_df,dfs[vent_df] )

time: 371 ms


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [59]:
dfs[weight_df]['uom']='kg'
dfs[weight_df].head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom,t_0
0,200001,61.0,61.000000,NaN,NaN,NaN,kg,2181-11-26
1,200003,77.0,77.000000,77.500000,72.57472,NaN,kg,2199-08-04
2,200012,51.2,51.200001,51.200001,NaN,NaN,kg,2153-12-23
3,200014,62.0,62.000000,NaN,NaN,NaN,kg,2105-02-17
4,200019,65.0,65.000000,NaN,NaN,NaN,kg,2178-07-09


time: 13.7 ms


In [60]:
dfs[weight_df]= dfs[weight_df][dfs[weight_df]['weight'].notnull()]
dfs[weight_df]= dfs[weight_df][['icustay_id','weight','uom']]

dfs[weight_df]['label']= 'weight'
dfs[weight_df]=dfs[weight_df].rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[weight_df]['delta']=pd.to_timedelta(0,'days')

#adding t_0
dfs[weight_df]= pd.merge(dfs[weight_df], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 128 ms


In [61]:
# pd.DataFrame(dfs[weight_df]).to_csv(Path(
#     save_path+'/%s_weight_prepped.csv' %(date)))
save_df(dfs[weight_df], 'weight')

del(dfs[weight_df])

time: 438 ms


In [62]:
dfs[height_df]['uom']='cm'

time: 3.1 ms


In [63]:
dfs[height_df]= dfs[height_df][dfs[height_df]['height'].notnull()]
dfs[height_df]= dfs[height_df][['icustay_id','height','uom']]

time: 9.15 ms


In [64]:
dfs[height_df]['label']= 'height'
dfs[height_df]=dfs[height_df].rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[height_df]['delta']=pd.to_timedelta(0,'days')
#adding t_0
dfs[height_df]= pd.merge(dfs[height_df], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 105 ms


In [65]:
#heightfirstday
save_df(dfs[height_df], 'height')
del(dfs[height_df])

time: 407 ms


# UTI

In [66]:
dfs[uti_df]['value'].unique()#seems good #all uti within clinical timewindow

array(['NEG', 'MOD', 'POS', 'TR', 'SM', 'LG', ' ', 'N', 'SMALL'],
      dtype=object)

time: 14.4 ms


In [67]:
dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='NEG')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows

dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='COMPUTER NETWORK FAILURE. TEST NOT RESULTED.')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows
dfs[uti_df]= dfs[uti_df].drop_duplicates(subset=['hadm_id','value','charttime'])

dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].notnull(),'value']= 1
dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].isna(),'value']= 0 

time: 170 ms


In [68]:
dfs[uti_df].head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
132,21,109451.0,51486,2134-09-12 00:08:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41071,2134-09-12,0 days 00:08:00.000000000
186,21,109451.0,51487,2134-09-12 00:08:00,1,NaN,NaN,Nitrite,Urine,Hematology,5802-4,41071,2134-09-12,0 days 00:08:00.000000000
236,31,128652.0,51486,2108-08-24 16:30:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,3453,2108-08-24,0 days 16:30:00.000000000
790,64,172056.0,51486,2143-03-03 04:33:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,03811,2143-03-03,0 days 04:33:00.000000000
832,62,116009.0,51486,2113-02-15 12:27:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,1125,2113-02-15,0 days 12:27:00.000000000


time: 23 ms


In [69]:
def uti_categorizer(uti_df):
    "useful to get all rows of days with positive values for patients (if multiple pos in a day there will be only 1 row for that day). ie more longitudinal format "
    
    #gives the max pos or neg value per day for 
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})  
    df_timewindow_perday= pd.merge(df_timewindow_perday,final_pt_df2[['icustay_id','hadm_id', time_var]], left_on='hadm_id', right_on='hadm_id', how='left')
    df_timewindow_perday=df_timewindow_perday.sort_values(['hadm_id','value','delta'], ascending=[True,False,True])
    return(df_timewindow_perday)

def uti_collapser(uti_df, label):
    "collapsing longitudinal data into 1 value. will return pos or neg if patient has a positive uti in their stay. one row per icustay_id"
    
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})
    first_pos= df_timewindow_perday.drop_duplicates(['hadm_id'])
    collapsed= pd.merge(final_pt_df2[['hadm_id','icustay_id','subject_id', time_var]],first_pos, left_on='hadm_id', right_on='hadm_id', how='left')
    
    collapsed['value']= collapsed['value'].fillna(0)
    collapsed.loc[collapsed.loc[:,'value']==1,'value']= 'pos'
    collapsed.loc[collapsed.loc[:,'value']==0,'value']= 'Neg/Not_tested'
    
    collapsed['delta']= collapsed['delta'].fillna(pd.Timedelta(1, unit='d'))
    
    collapsed['label']= label
    
    collapsed['uom']='pos/neg category'
    return(collapsed)

time: 47.4 ms


In [70]:
uti_nit_pos= dfs[uti_df][dfs[uti_df]['label']=="Nitrite"]
uti_leuk_pos= dfs[uti_df][dfs[uti_df]['label']=="Leukocytes"]

leuk_collapsed= uti_collapser(uti_leuk_pos, 'leukocyte')
nit_collapsed= uti_collapser(uti_nit_pos, 'nitrite')


time: 101 ms


In [71]:
save_df(leuk_collapsed, 'leuk')
del(leuk_collapsed)

save_df(nit_collapsed, 'nit')
del(nit_collapsed)
del(dfs[uti_df])

time: 603 ms


# bloodgas

In [27]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]
dfs[bg_df] = dfs[bg_df].rename(index=str, columns={'valueuom':'uom'})
dfs[bg_df]= dfs[bg_df].loc[~(dfs[bg_df].loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
dfs[bg_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
48,9,150750,220597,2149-11-10 03:04:00,BASEEXCESS,NaN,0,mEq/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
49,9,150750,220597,2149-11-10 03:04:00,CALCIUM,1.14,1.14,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
50,9,150750,220597,2149-11-10 03:04:00,FIO2,60.00,60,NaN,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
51,9,150750,220597,2149-11-10 03:04:00,LACTATE,2.40,2.4,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
52,9,150750,220597,2149-11-10 03:04:00,PCO2,41.00,41,mm Hg,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000


time: 1.58 s


### most likely erroneous value removal
the code below manually removes values that are a high likelyhood of being erroneous in a way that doesn't follow great programming practices. on my list of things to improve, but it is currently functional. 


In [28]:
##calcium
#fixing the calcium errors w/o hard coding
(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 679 ms


In [29]:
##min chloride
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'value'])=np.nan

time: 463 ms


In [30]:
#peep changes summary: 
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='PEEP')&
                        (dfs[bg_df]['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 66.1 ms


In [31]:
#temp changes summary:

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'valuenum']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'valuenum']= np.nan

time: 483 ms


In [32]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='0'),'value']=np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum']

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 771 ms


#### removing null values annotated abov

In [33]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]

time: 142 ms


### splitting specimen out for vent vs non-vent bg data

In [34]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= dfs[bg_df].loc[dfs[bg_df].loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

dfs[bg_df]= pd.merge(dfs[bg_df],specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary=dfs[bg_df].loc[dfs[bg_df].loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 1.54 s


In [35]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE','FIO2']
bg_vent_labels=['PCO2','PaO2','PO2','PEEP','O2FLOW']

time: 1.51 ms


In [36]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df].loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label'].isin(bg_vent_labels),:]

time: 306 ms


In [37]:
dfs[bg_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta,specimen
1,9,150750,220597,2149-11-10 03:04:00,CALCIUM,1.14,1.14,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
2,9,150750,220597,2149-11-10 03:04:00,FIO2,60.00,60,NaN,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
3,9,150750,220597,2149-11-10 03:04:00,LACTATE,2.40,2.4,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
5,9,150750,220597,2149-11-10 03:04:00,PH,7.41,7.41,units,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
10,9,150750,220597,2149-11-10 21:57:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-10 21:57:00,2149-11-10,0 days 21:57:00.000000000,ART


time: 30.1 ms


In [38]:
bg_col=['subject_id','hadm_id','icustay_id','charttime','delta',time_var,'label','valuenum','uom']
bg_ART_nosummary= bg_ART_nosummary[bg_col]
dfs[bg_df]= dfs[bg_df][bg_col]
del(bg_col)

time: 51.6 ms


In [39]:
bg_ART_nosummary['label'].unique()

array(['PCO2', 'PaO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 9 ms


In [40]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow', time_var=time_var)
o2_flow_df['label']="o2_flow" #fixing label

#removing o2_flow from bg_ART
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='O2FLOW',:]

time: 262 ms


In [41]:
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='PEEP',:]

time: 34.5 ms


## PaO2:FiO2
* PaO2: measurement of oxygen pressure in arterial blood
* FiO2: % of oxygen in the air a patient is breathing. in normal air this is 21% oxygen
* SpO2: Peripheral capillary oxygen saturation, estimate of the amount of oxygen in the blood. this is % of oxygenated haemoglobin compared to total haemoglobin

***requirements: run vitals, ventilation before this to get spo2 & ventilation category.
I would modify the criteria to be more strict:
* first isolate patients on mech or oxygen ventilation: these are only patients we calc P:F for
* if no PaO2:
 * find SpO2 <=97 --> ~PaO2
   * SpO2 to estimate and PaO2 equivalent via the equation used by Knox et al to convert to PaO2-equivalent.
* once have all PaO2 and estimated PaO2:
 * find fio2 measured between t- 6hours :t, where t= time of PaO2  
 * use most recent fio2 measure in this time?
* Use P:F as a continuous value unless there is a clear need to bucket, and if so use <100,100-200, 200-300, >300

##### first isolate patients who had mech or oxygen ventilation during their N hours
these are only patients we calc P:F for

In [42]:
def time_window_filter(df, final_pt_df2,timecol,upper_window, lower_window, time_var='t_0'):
    """
    will take in any df and filter to only values between lower_window and upper_window. 
    values = the difference between timecol-timevar. 
        most commonly this is charttime- t_0 (or first in ICU ABrx meeting criteria)
    will add delta and t_0 to df as well which are used lateron.git
    """
    #global upper_window, lower_window 
    
    try:
        df= pd.merge(df, final_pt_df2[['icustay_id',time_var]], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    return(df_after_t0)

time: 66.3 ms


In [44]:
pao2_test_df= time_window_filter(pao2_test_df,
                                       final_pt_df2,
                                       time_col,
                                       time_var=time_var,
                                       lower_window=lower_window,
                                       upper_window=upper_window
                                      )

pao2_test_df['label']='PaO2'
pao2_test_df['uom']='mm Hg'

time: 294 ms


In [45]:
import config 
import psycopg2
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

fio2_chart =query_schema +"""---------- collecting fio2 from chartevents
with stg_fio2 as 
(
  select SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME
    -- pre-process the FiO2s to ensure they are between 21-100%
    , max(
        case
          when itemid = 223835
            then case
              when valuenum > 0 and valuenum <= 1
                then valuenum * 100
              -- improperly input data - looks like O2 flow in litres
              when valuenum > 1 and valuenum < 21
                then null
              when valuenum >= 21 and valuenum <= 100
                then valuenum
              else null end -- unphysiological
        when itemid in (3420, 3422)
        -- all these values are well formatted
            then valuenum
        when itemid = 190 and valuenum > 0.20 and valuenum < 1
        -- well formatted but not in %
            then valuenum * 100
      else null end
    ) as value
  from mimiciii.CHARTEVENTS
  where ITEMID in
  (
    3420 -- FiO2
  , 190 -- FiO2 set
  , 223835 -- Inspired O2 Fraction (FiO2)
  , 3422 -- FiO2 [measured]
  )
  -- exclude rows marked as error
  and error IS DISTINCT FROM 1
  group by SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME )
  
  SELECT * from stg_fio2"""

fio2_chart_df = pd.read_sql_query(fio2_chart,conn) #38637

#restricts to cohort
fio2_chart_df=fio2_chart_df[fio2_chart_df['icustay_id'].isin(icustay_id)]



time: 32 s


In [46]:
#restricting fio2 from chartevents to relevant clinical timewindow. 
fio2_chart_df= time_window_filter(fio2_chart_df,
                                       final_pt_df2,
                                       time_col,
                                       time_var=time_var,
                                       lower_window=lower_window,
                                       upper_window=upper_window
                                      )

time: 402 ms


first, filter pao2 and fio2 to only icustay who have ventilation (pf_ratio_icu)

In [47]:
vent_icu=list(ventcategory_1day_df[ventcategory_1day_df['value'].isin(['Mech'])]['icustay_id'].unique())

time: 8.14 ms


next isolate the pao2 and fio2 from the bg arterial dataframe. 
 * pao2: all pao2 measurements in cohort -> pao2 measurements for icustay with ventilation
 * fio2: all fio2 measurements in cohort -> fio2 measurements for icustay with ventilation

In [48]:
###using the pao2 sql
fio2_chart_df['delta']=pd.to_timedelta(fio2_chart_df['delta'])
fio2_chart_df= fio2_chart_df[fio2_chart_df['value'].notnull()].copy()

pao2_vent= pao2_test_df[pao2_test_df.loc[:,'icustay_id'].isin(vent_icu)].copy()
pao2_vent['delta']=pd.to_timedelta(pao2_vent['delta']).copy()
pao2_vent= pao2_vent[pao2_vent['value'].notnull()] #removing some null rows

time: 184 ms


In [49]:
#using bloodgas

# fio2_chart_df['delta']=pd.to_timedelta(fio2_chart_df['delta'])
# fio2_chart_df= fio2_chart_df[fio2_chart_df['value'].notnull()].copy()

# pao2= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='PaO2',:].copy() 
# pao2.rename(index=str, columns={'valuenum':'value'}, inplace=True)
# pao2_vent= pao2[pao2.loc[:,'icustay_id'].isin(vent_icu)].copy()
# pao2_vent['delta']=pd.to_timedelta(pao2_vent['delta']).copy()
# pao2_vent= pao2_vent[pao2_vent['value'].notnull()] #removing some null rows

time: 2.38 ms


##### next: find patients with ventilation but no PaO2 and collect their SpO2 values

In [50]:
# pao2_icu: icustay_id for pao2 with vent
pao2_icu= list(pao2_vent['icustay_id'].unique())

# find people with ventilation but no PaO2 measured (ie those who need it approximatd)
vent_but_no_pao2_icu=set(vent_icu).difference(set(pao2_icu))

## if on vent but no PaO2, then use SpO2 (if 97 or less) -> estimate PaO2 via equation below
spo2_filtered= spo2[(spo2['icustay_id'].isin(vent_but_no_pao2_icu)) &
                    (spo2['value']<=97.0)]

time: 175 ms


In [51]:
##most updated
print(
    "total icustay:",  len(icustay_id), '\n',
    'mech during their 72 hours of clinical data: ',  len(vent_icu), '\n',
    '   -pao2, +vent: ',  pao2_vent['icustay_id'].nunique(), '\n',
    '   +fio2, +vent: ',  fio2_chart_df[fio2_chart_df['icustay_id'].isin(vent_icu)]['icustay_id'].nunique(), '\n',
    
    '+ vent, - pao2: ',  len(vent_but_no_pao2_icu), '\n',
    '+fio2, +vent, -pao2: ',  fio2_chart_df[fio2_chart_df['icustay_id'].isin(vent_but_no_pao2_icu)]['icustay_id'].nunique(), '\n',
)

total icustay: 19633 
 mech during their 72 hours of clinical data:  10156 
    -pao2, +vent:  6022 
    +fio2, +vent:  10037 
 + vent, - pao2:  4134 
 +fio2, +vent, -pao2:  4094 

time: 51.2 ms


##### next: estimate PaO2 using SpO2 for patients who have ventilation but no PaO2 measurement 
$$\text{Ellis Severinghaus inversion}: PaO2_{estimate} =\sqrt[3]{\frac{1}{2} (-y_N + \sqrt{y_N^2 -h^2})} + \sqrt[3]{\frac{1}{2} (-y_N - \sqrt{y_N^2 -h^2})}$$

$$ \begin{cases} 
      h^2 =-500000 \\
      y_N = -23400 * \frac{s}{1-s} \\
   \end{cases}
$$
* s is the fractional oxygen saturation (0–1) and p is the associated oxygen tension in mm Hg. 
* qc: when s=0.5, p=26.856
* equation source: http://www.nickalls.org/dick/papers/anes/severinghaus.pdf


In [52]:
def Ellis_SpO2(S):
    """
    ellis inversion equation to approximate PaO2 from SpO2. 
    where s is the fractional oxygen saturation (SpO2) (0–1) and p is the associated oxygen tension in mm Hg (PaO2). 
    """
    if S >1 and S<100:
        S=S/100.
    elif S==1:
        S=0.9999
    elif S>0 and S<1:
        pass

    h2= -500000.
    yn= -23400. * (S/(1.-S))

    term1= np.sqrt(np.power(yn,2.) - h2)

    term2= 0.5*(-yn + term1)
    term3= 0.5*(-yn - term1)

    term4= np.sign(term2) * np.abs(term2)**(1/3)
    term5= np.sign(term3) * np.abs(term3)**(1/3)
    p= term4+term5
    return(p)
    

time: 36.4 ms


In [53]:
spo2_filtered= spo2[(spo2['icustay_id'].isin(vent_but_no_pao2_icu)) &
                    (spo2['value']<=97.0)]

time: 181 ms


In [54]:
# approximating Pao2 from SpO2:

## only usable for spo2 values <=97
spo2_filtered= spo2[(spo2['icustay_id'].isin(vent_but_no_pao2_icu)) &
                    (spo2['value']<=97.0)]

# run the Ellis_SpO2 to approx PaO2.
spo2_filtered.loc[:,'value']= spo2_filtered.value.apply(Ellis_SpO2) #note this produces a false positive SettingWithCopyWarning

spo2_filtered.loc[:,'label']='PaO2_estimate'
spo2_filtered.loc[:,'uom']='mm Hg estimate'


time: 2.11 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [55]:
#append the pao2 (for those with ventilation) and new approximated pao2 together
pao2_appended=pao2_vent.append(spo2_filtered, sort=False).copy()

time: 699 ms


* once have all PaO2 and estimated PaO2:
 * find fio2 measured between t- 6hours :t, where t= time of PaO2  
 * use most recent fio2 measure in this time?

In [56]:
## find all associated fio2 for pao2 that occured between 6hr before and time of PaO2.
###make fio2 look back period a full 6 hours, use the most recent.
#need to groupby icustay_id and charttime for both. pan

#quick time conversion to make sure both in timedelta format
pao2_appended['delta']=pd.to_timedelta(pao2_appended['delta'])
pao2_appended['charttime']=pd.to_datetime(pao2_appended['charttime'])
fio2_chart_df['delta']=pd.to_timedelta(fio2_chart_df['delta'])
fio2_chart_df['charttime']=pd.to_datetime(fio2_chart_df['charttime'])
fio2_chart_df['label']='fio2'


#labeling all PaO2 (both measured and approx) with offset column= the delta time offset by the n hours of time we look back for Fio2 values
pao2_appended['offset']=pao2_appended['charttime'] - pd.to_timedelta(6, unit='h') #find most recent in this. 

time: 1.33 s


In [57]:
#left cartesian product of pao2 and fio2 measurements. 
pao2_fio2_windowed= pd.merge(pao2_appended,
              fio2_chart_df[['icustay_id','value','label','charttime','delta']].rename(index=str, columns={'value':'value2','label':'label2','charttime':'charttime2','delta':'delta2'}),
              left_on='icustay_id',
              right_on='icustay_id',
              how='left') 

##take this left cartesian product and filter to rows where charttime2 (ie fio2 time) is between t-6:t where t= time at pao2 measurement. 
#now this should be all Pao2 (real and estimated) and Fio2 combinations that 
pao2_fio2_windowed= pao2_fio2_windowed[pd.to_datetime(pao2_fio2_windowed['charttime2']).between(pd.to_datetime(pao2_fio2_windowed['offset']),pd.to_datetime(pao2_fio2_windowed['charttime']))]

time: 1min 35s


In [58]:
## grabs the most recent FiO2 value for each associated PaO2 
ratio_df= pao2_fio2_windowed.sort_values(['icustay_id','charttime','charttime2'], ascending=[True,True, False]).drop_duplicates(['icustay_id','charttime'],keep='first')

time: 4.03 s


In [59]:
ratio_df.loc[ratio_df.loc[:,'value2'].isnull(),'value2']= 21 #impute 21 for FiO2 values that are missing (very few of them)
ratio_df['ratio']= ratio_df['value']/(ratio_df['value2']/100) #calculate the ratio
ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime2']= ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime']
ratio_df['deltadelta']=pd.to_datetime(ratio_df['charttime'])-pd.to_datetime(ratio_df['charttime2'])

time: 1.27 s


In [61]:
#rerunning the algo #2 with pao2 sql.
ratio_df['ratio'].describe()

count    154538.000000
mean        183.063291
std          87.185763
min           3.753891
25%         132.857143
50%         170.000000
75%         222.000000
max        1666.666600
Name: ratio, dtype: float64

time: 16.6 ms


In [325]:
#using my prior values
ratio_df['ratio'].describe()

count    44410.000000
mean       185.415388
std        103.817888
min          9.192131
25%        116.997304
50%        167.000000
75%        226.432633
max       2376.190476
Name: ratio, dtype: float64

time: 7.56 ms


In [62]:
ratio_df[['icustay_id','ratio']].groupby('icustay_id')['ratio'].apply(max).describe()

count    9589.000000
mean      300.502896
std       154.408614
min        33.347852
25%       204.999992
50%       254.999996
75%       362.500000
max      1666.666600
Name: ratio, dtype: float64

time: 2.28 s


# factorizing pco2 start 

In [110]:

# yn_convert_band(max_bands, #df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_pco2(df=max_bands,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    yn_df = pd.merge(pt[['icustay_id','hadm_id','subject_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 46.1 ms


In [111]:
bg_ART_nosummary= bg_ART_nosummary.rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 15.1 ms


In [112]:
pco2_df=bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2']
max_bands=pco2_df.loc[pco2_df.groupby('icustay_id', as_index=False)['value'].idxmax(),:]
del(pco2_df)

time: 1.51 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [113]:
pco2_cat=yn_convert_band(df=max_bands,           
                    label_fill="PCO2",
                    threshold=50,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 142 ms


In [114]:
#drop bands from lab_df
bg_ART_nosummary=bg_ART_nosummary.drop(bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2'].index)


time: 23 ms


In [115]:
##dropping charttime, may be problematic later. 06/13/19
bg_ART_nosummary=bg_ART_nosummary.drop('charttime', axis=1)

time: 1.81 ms


In [116]:
bg_ART_nosummary=bg_ART_nosummary.append(pco2_cat, sort=True)

time: 7.37 ms


In [117]:
bg_ART_nosummary.loc[bg_ART_nosummary['label']=='PCO2','value'].value_counts()

absent    11981
<50        5385
>50        2267
Name: value, dtype: int64

time: 8.41 ms


# factorizing bands end 

In [118]:
save_df(ratio_min_df,'pafaRatio') # minimum pao2:fio2 ratio during a stay
del(ratio_min_df)

save_df(dfs[bg_df],'bg_all') #all bloodgas
del(dfs[bg_df])

save_df(bg_ART_nosummary,'bg_ART') #only ARTERIAL bloodgas
del(bg_ART_nosummary)

save_df(o2_flow_df,'o2_flow') #need to investigate this more
del(o2_flow_df)

time: 3.33 s


# RRT

In [119]:
#removing null values
dfs[rrt_df] = dfs[rrt_df].loc[dfs[rrt_df]['rrt'].notnull(),:]
dfs[rrt_df]['uom']='category'
dfs[rrt_df]['delta']=pd.to_timedelta(
    pd.to_datetime(dfs[rrt_df]['first_charttime'])-
    pd.to_datetime(dfs[rrt_df][time_var]),
    'days')
dfs[rrt_df]['label']= 'rrt'
dfs[rrt_df]=dfs[rrt_df].rename(index=str, columns={'rrt':'value'})

time: 43 ms


### converting to yes/no

In [120]:
dfs[rrt_df].head()

,icustay_id,t_0,first_charttime,value,uom,delta,label
0,294638,2191-03-16,NaN,0.0,category,NaT,rrt
1,220597,2149-11-10,NaN,0.0,category,NaT,rrt
2,232669,2104-08-11,NaN,0.0,category,NaT,rrt
3,273430,2108-08-06,NaN,0.0,category,NaT,rrt
4,217847,2134-09-12,2134-09-12 13:30:00,1.0,category,13:30:00,rrt


time: 12.6 ms


In [121]:
dfs[rrt_df]['icustay_id'].nunique() #

19633

time: 4.11 ms


In [122]:
rrt_yn= yn_convert(dfs[rrt_df],label_fill='rrt', time_var=time_var)

time: 48.2 ms


In [123]:
save_df(rrt_yn, 'rrt')
del(rrt_yn, dfs[rrt_df])

time: 464 ms


# GCS_72

In [124]:
dfs[gcs_df]['uom']='GCS_score'

time: 1.25 ms


In [125]:
list(dfs[gcs_df])

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'icu_admit',
 't_0',
 'approx_charttime',
 'admit_plus_day',
 'delta',
 'uom']

time: 2.61 ms


In [126]:
dfs[gcs_df]['label']= 'mingcs'
dfs[gcs_df]['uom']='gcs_score'
dfs[gcs_df]=dfs[gcs_df].rename(index=str, columns={'mingcs':'value'})

time: 14.4 ms


In [127]:
dfs[gcs_df]=dfs[gcs_df][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 3.46 ms


In [128]:
save_df(dfs[gcs_df], 'gcs')
del(dfs[gcs_df])

time: 56.6 ms


# SOFA
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [129]:
dfs[sofa_df]['uom']='daily_sofa_score'

#adding day delta column
dfs[sofa_df]=dfs[sofa_df].sort_values(['hadm_id','day',time_var]) #good
dfs[sofa_df]['day_rank']=dfs[sofa_df].groupby('icustay_id')['day'].rank()
dfs[sofa_df]['delta']=pd.to_timedelta((dfs[sofa_df]['day_rank']-1), 'days')

time: 81.2 ms


In [130]:
dfs[sofa_df]['label']= 'daily_sofa'
dfs[sofa_df]=dfs[sofa_df].rename(index=str, columns={'sofa':'value'})

time: 145 ms


In [131]:
dfs[sofa_df]= dfs[sofa_df][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 16.2 ms


In [132]:
dfs[sofa_df].head()

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
88189,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
52007,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
52008,23018,100007,217937,1 days,daily_sofa,4,2145-03-31,daily_sofa_score
52009,23018,100007,217937,2 days,daily_sofa,0,2145-03-31,daily_sofa_score
1346,533,100009,253656,0 days,daily_sofa,5,2162-05-17,daily_sofa_score


time: 12.9 ms


In [133]:
save_df(dfs[sofa_df], 'sofa')
del(dfs[sofa_df])

time: 1.12 s


# patient Demographic variables

In [134]:
dfs[pt_info_df]

,icustay_id,subject_id,t_0,label,value,delta,uom
0,280836,268,2198-02-16,yearsold,65.9848802824042,0 days 00:00:00.000000000,yearsold
1,206613,269,2170-11-05,yearsold,40.0998279019951,0 days 00:00:00.000000000,yearsold
2,219649,275,2170-10-08,yearsold,82.164211885568,0 days 00:00:00.000000000,yearsold
3,204407,279,2164-06-15,yearsold,74.2940118069815,0 days 00:00:00.000000000,yearsold
4,257572,281,2101-10-18,yearsold,60.0142318173752,0 days 00:00:00.000000000,yearsold
5,238023,285,2152-09-22,yearsold,45.3551008314954,0 days 00:00:00.000000000,yearsold
6,260225,286,2176-01-01,yearsold,85.0156237166324,0 days 00:00:00.000000000,yearsold
7,278519,290,2179-01-27,yearsold,74.3841495867873,0 days 00:00:00.000000000,yearsold
8,291141,306,2199-09-16,yearsold,60.9386263847694,0 days 00:00:00.000000000,yearsold
9,265862,307,2162-11-28,yearsold,74.7267591958831,0 days 00:00:00.000000000,yearsold


time: 30.4 ms


In [135]:
##adjusting ages over 90 (which were set to 300 to deidentify) to 90
admit_index=dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')].index
age_tf=pd.to_numeric(dfs[pt_info_df].loc[admit_index,'value'])>90
dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')&(age_tf),'value']=90.0

time: 46.4 ms


In [136]:
dfs[pt_info_df].loc[admit_index,'value'].value_counts()

90.0                776
63.4081637069993      1
66.5478703386823      1
61.6716203703704      1
49.2782088942125      1
44.4676982406774      1
47.3690400410678      1
85.5589544515426      1
82.7865793976728      1
73.6585454217051      1
65.4810862359622      1
61.153896145461       1
52.8789559408827      1
72.2622747927599      1
44.1619576900652      1
55.8491155854691      1
86.4934200953178      1
77.1439961213781      1
50.9048232755343      1
49.2291726557153      1
76.2256356947296      1
77.8772847745076      1
61.3304719306918      1
73.8797097054275      1
65.1581717240855      1
68.1677385479251      1
20.758970105458       1
88.5625411628261      1
88.568174449261       1
77.1119210586356      1
                   ... 
78.9128267675615      1
51.0118879128958      1
51.7122149656501      1
66.2069385821482      1
56.8568453874819      1
76.708561265749       1
58.239348651355       1
49.4923419715061      1
75.525660348062       1
50.5303281618374      1
71.7062315258448

time: 13.2 ms


In [137]:
#dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='first_admit_age') & (dfs[pt_info_df]['value']>90)]

time: 480 µs


In [138]:
#date= '22102018'
save_df(dfs[pt_info_df], 'pt_info')

del(dfs[pt_info_df] )

time: 261 ms
